In [3]:
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import json
import gc
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [4]:
import pandas as pd

df = pd.read_csv(r'C:\Users\hp\Desktop\sem6\PL Lab\leb11-OEL\app\train_data.csv')

In [5]:
df['encoded_text'] = df['label'].astype('category').cat.codes
df.head()

,sentence,label,encoded_text
0,Highlights\nuMaersk’s Q2 2023 EBITDA of USD2.9...,negative,0
1,Highlights\nuWe expect 2023 total sales to con...,positive,1
2,"Highlights\nuIn 2023, we expect revenue growth...",positive,1
3,Highlights\nuTight supply conditions hurt AEIS...,positive,1
4,Highlights\nuAena’s Q2 2023 EPS of EUR3.16 bea...,positive,1


In [6]:
data_texts = df['sentence'].to_list()
data_labels = df['encoded_text'].to_list()

In [7]:
print(data_labels)

[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1]


In [8]:
print(data_texts)

['Highlights\nuMaersk’s Q2 2023 EBITDA of USD2.9 billion (-\n72% Y/Y; -27% Q/Q) was 16% above consensus \nestimate, attributed to good cost management \ninstead of strong rates, in our view. The fall in \nearnings was mainly caused by Ocean (-76% Y/\nY; -33% Q/Q) on reduced freight rates (-51% Y/\nY; -15% Q/Q) and weaker volumes (-6% Y/Y; \n+7% Q/Q). \nuMaersk lifted the bottom end of its 2023 EBITDA \nguidance to USD9.5-USD11.0 billion from \nUSD8.0-11.0 billion reflecting the stronger H1 \n2023 results but lowered its 2023 market \nvolume outlook to -4.0% to -1.0% (previously -\n2.5% to +0.5%) on continued inventory \ncorrection till the end of 2023. We think the \nimplied sequential drop in H1 2023 earnings \nreflects the ongoing decline in Maersk’s \ncontracted rate given that they are being reset \nto a level nearer to the spot which has fallen \nsharply since mid-2022. Also, we gathered that \nthe industry’s supply of new vessels may \nincrease freight capacity by 9% and 10% in \

In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0)

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation = True, padding = True  )
val_encodings = tokenizer(val_texts, truncation = True, padding = True )

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [12]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [14]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments


training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=7,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    eval_steps=100                   
)

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 5 )


trainer = TFTrainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)

ImportError: cannot import name 'TFTrainer' from 'transformers' (c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\__init__.py)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
save_directory = r"C:\Users\hp\Desktop\preply\lars\project\module 4\testing" 
model.save_pretrained(save_directory)
toknizer.save_pretrained(save_directory)

In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

In [ ]:
test_text = test_texts[0]

test_text

In [ ]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'    
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

In [ ]:
import torch

from transformers import DistilBertForSequenceClassification

tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)


model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True )
predict_input_pt = tokenizer_fine_tuned_pt(test_text, truncation = True, padding = True, return_tensors = 'pt' )

ouput_pt = model_fine_tuned_pt(predict_input_pt)

prediction_value_pt = torch.argmax(ouput_pt[0], dim = 1 ).item()

prediction_value_pt